# Machine Learning

## Исследовательский анализ данных

Продолжаем исследовать данные из crops_usa.csv.

Во-первых, стоит проверить содержание столбцов Year и State — действительно ли в датасете представлены данные за все 40 лет и по всем штатам.

И, во-вторых, построить графики по существующим столбцам. Например, интересно посмотреть, насколько штаты различаются по посевным площадям и объёму урожая. Лучше всего это позволит выявить столбчатая диаграмма.

Но прежде чем строить диаграмму, познакомьтесь с новыми инструментами.

In [ ]:
# импортируйте библиотеку pandas
import pandas as pd
# прочитайте файл и сохраните его в переменной data
data=pd.read_csv("crops_usa.csv")
# выведите data на экран
data.head() # main data

In [ ]:
states = data["State"].unique() # list of unique entries
print(states)

states_unique_count = data['State'].nunique() # count of unique entries
print(states_unique_count)

# Логическая индексация в столбце

В прошлой теме вы научились отбирать нужные значения из столбцов по их порядковому номеру, или индексу. Так вы выбирали недели рекламных кампаний. А что делать, если мы хотим отобрать данные за определённый год, но порядковых номеров не знаем? Выход — добавить условие. Такой приём в pandas называют логической индексацией.
Покажем на практике. Выберем все значения из столбца Production, для которых год (столбец Year) равен 2018:

In [ ]:
production_2018 = data[data['Year'] == 2018]['Production']
print(production_2018)

years_unique = data['Year'].unique()

# количество уникальных значений столбца Year
years_unique_count = data['Year'].nunique()

print(years_unique)
print(years_unique_count)

Постройте столбчатую диаграмму, отражающую площадь посевов для каждого штата за 2019 год. По оси X разместите значения переменной acres_2019, а по оси Y — states_2019.

In [ ]:
import matplotlib
import seaborn

matplotlib.rcParams['figure.figsize'] = [10, 7] # указываем размер графика

acres_2019 = data[data['Year'] == 2019]['Acres']
states_2019 = data[data['Year'] == 2019]['State']

In [ ]:
# значения столбца Production за 2019 год
production_2019 = data[data['Year'] == 2019]['Production']
states_2019 = data[data['Year'] == 2019]['State']


# постройте столбчатую диаграмму
seaborn.barplot(x=production_2019, y=states_2019)

In [ ]:
# 1) группируем данные по году (Year) с помощью groupby('Year')
# 2) достаём из сгруппированной таблицы только столбец с площадью посевов
# 3) суммируем значения в каждой группе с помощью sum()
acres_usa = data.groupby('Year')['Acres'].sum()
print(acres_usa)
production_usa = data.groupby('Year')['Production'].sum()

# Делим значения столбца production_usa на значения столбца acres_usa,
# получаем урожайность и сохраняем её в yield_usa
yield_usa = production_usa / acres_usa

# достаём индексы (порядковые номера) столбца acres_usa, это числа от 1980 до 2019
years_unique = acres_usa.index.values

# постройте столбчатую диаграмму урожайности по годам
seaborn.barplot(x=years_unique, y= yield_usa)


На этапе уточнения вы сформулировали задачу так: «Определить, что даст более точный прогноз объёма урожая: предсказание площади посевов или урожайности».
Объём урожая (production) за определённый год (t) вычисляется как площадь засеянных в этом году полей (acres) умножить на урожайность (yield) за тот же год:
​$$production_t = acres_t \cdot yield_t$$
 
Самый точный прогноз получится, если предсказать и площадь посевов, и урожайность. Однако Герман может предсказать только один фактор, а второй планирует взять из данных за предыдущий год. Значит, есть две стратегии:
предсказать площадь посевов, а урожайность взять за предыдущий год;
предсказать урожайность, а площадь посевов взять за предыдущий год.
Какая даст более точный прогноз урожая? Ответив на этот вопрос, вы решите задачу.

In [ ]:
# группируем значения Acres и Production по годам и суммируем по каждой группе
acres_usa = data.groupby('Year')['Acres'].sum()
production_usa = data.groupby('Year')['Production'].sum()

# Делим значения столбца production_usa на значения столбца acres_usa,
# получаем урожайность и сохраняем её в yield_usa
yield_usa = production_usa / acres_usa

# Умножьте площадь посевов за каждый год
# на урожайность за предыдущий год,
# получив её с помощью команды shift(1)
predict_acres = acres_usa * yield_usa.shift(1)
print(predict_acres)

In [ ]:
# группируем значения Acres и Production по годам и суммируем по каждой группе
acres_usa = data.groupby('Year')['Acres'].sum()
production_usa = data.groupby('Year')['Production'].sum()

# Делим значения столбца production_usa на значения столбца acres_usa,
# получаем урожайность и сохраняем её в yield_usa
yield_usa = production_usa / acres_usa

# Умножаем площадь посевов за каждый год
# на урожайность за предыдущий год
predict_acres = acres_usa * yield_usa.shift(1)
print(predict_acres)

# пустая строка, чтобы разделить два результата
print()

# Умножьте урожайность за каждый год
# на площадь посевов за предыдущий год,
# получив её с помощью команды shift(1)
predict_yield = acres_usa.shift(1) * yield_usa
print(predict_yield)

In [ ]:

# группируем значения Acres и Production по годам и суммируем по каждой группе
acres_usa = data.groupby('Year')['Acres'].sum()
production_usa = data.groupby('Year')['Production'].sum()

# Делим значения столбца production_usa на значения столбца acres_usa,
# получаем урожайность и сохраняем её в yield_usa
yield_usa = production_usa / acres_usa

# Первая стратегия:
# умножим площадь посевов за каждый год
# на урожайность за предыдущий год (то есть со сдвигом на 1)
predict_acres = acres_usa * yield_usa.shift(1)

# ошибка — это разность между реальными значениями и предсказанными
error_acres = production_usa - predict_acres
# Удаляем самое первое значение методом dropna(),
# оно не понадобится, ведь данных за 1979 год нет,
# и разность находить не с чем
error_acres = error_acres.dropna()

# достаём индексы столбца error_acres — теперь это числа с 1981 до 2019
years_numbers = error_acres.index.values

seaborn.barplot(x=years_numbers, y=error_acres)

In [ ]:
# группируем значения Acres и Production по годам и суммируем по каждой группе
acres_usa = data.groupby('Year')['Acres'].sum()
production_usa = data.groupby('Year')['Production'].sum()

# Делим значения столбца production_usa на значения столбца acres_usa,
# получаем урожайность и сохраняем её в yield_usa
yield_usa = production_usa / acres_usa

# Вторая стратегия:
# площадь посевов прошлогодняя (то есть со сдвигом на 1),
# а урожайность настоящая
predict_yield = acres_usa.shift(1) * yield_usa

# ошибка — это разность между реальными значениями и предсказанными
error_yield = production_usa - predict_yield
# Удаляем самое первое значение методом dropna(),
# оно не понадобится, ведь данных за 1979 год нет,
# и разность находить не с чем
error_yield = error_yield.dropna()

# достаём индексы столбца error_acres — это числа с 1981 до 2019
years_numbers = error_yield.index.values

# постройте столбчатую диаграмму
seaborn.barplot(x=years_numbers, y=error_yield)